In [1]:
import requests
import pandas as pd
import numpy as np
!pip install mysql-connector-python
import mysql.connector
from io import StringIO
import os


In [2]:
url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/liste-des-marches-de-la-collectivite-parisienne/exports/csv?lang=fr&timezone=Europe%2FHelsinki&use_labels=true&delimiter=%3B"
response = requests.get(url)
if response.status_code==200:
    csv_data = StringIO(response.text)
else:
    raise Exception("Could not download the file.")
    
df = pd.read_csv(csv_data, delimiter=';')
# Clean and format data, replacing NaN and empty strings with None
df = df.replace({np.nan: None, '': None})
df = df.applymap(lambda x: None if x in [np.nan, ''] else x)

In [3]:

#print(df.head())

In [4]:
connection = mysql.connector.connect(
    host=os.getenv('MYSQL_HOST', 'localhost'),  # Default to 'localhost'
    port=int(os.getenv('MYSQL_PORT', 3306)),  # Default to 3306
    user=os.getenv('MYSQL_USER', 'root'),  # Default user
    password=os.getenv('MYSQL_PASSWORD', 'password'),  # Default password
    database='france',
    charset='utf8mb4'
  
    
)

cursor = connection.cursor()
#cursor.execute("SHOW TABLES;")
#for table in cursor.fetchall():
    #print(table)
cursor.execute("DROP TABLE IF EXISTS markets")

table_creation_query = """

CREATE TABLE IF NOT EXISTS markets (
    id INT AUTO_INCREMENT PRIMARY KEY,
    annee_de_notification YEAR,
    numero_marche VARCHAR(50),
    objet_du_marche TEXT,
    nature_du_marche VARCHAR(255),
    fournisseur_nom VARCHAR(255),
    fournisseur_siret VARCHAR(20),
    fournisseur_code_postal VARCHAR(20),
    fournisseur_ville VARCHAR(255),
    montant_min DECIMAL(15, 2),
    montant_max DECIMAL(15, 2),
    date_de_notification DATE,
    date_de_debut DATE,
    date_de_fin DATE,
    duree_du_marche INT,
    perimetre_financier VARCHAR(255),
    categorie_d_achat_cle VARCHAR(255),
    categorie_d_achat_texte VARCHAR(255)
);
"""
cursor.execute(table_creation_query)
cursor.execute("DESCRIBE markets;")
for row in cursor.fetchall():
    print(row)
    
df.columns = (
    df.columns
    .str.strip()
    .str.replace(r'\s*-\s*', '_', regex=True)  # Replace hyphen with underscores
    .str.replace('é', 'e')  # Replace accented letters
    .str.replace('è', 'e')  # Replace other accented letters
    .str.replace("'", '_')  # Replace apostrophes with underscores
    .str.replace(' ', '_')  # Replace spaces with underscores
    .str.replace(r'\(.*?\)', '', regex=True)  # Remove text in parentheses
    .str.replace(r'_+$', '', regex=True)  # Remove trailing underscores
    .str.lower()
)
print(df.columns)
print(df.dtypes)

for col in df.select_dtypes(include=[object]).columns:
    df[col] = df[col].apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8') if isinstance(x, str) else x)
    
    
df['date_de_notification'] = pd.to_datetime(df['date_de_notification'], errors='coerce')
df['date_de_debut'] = pd.to_datetime(df['date_de_debut'], errors='coerce')
df['date_de_fin'] = pd.to_datetime(df['date_de_fin'], errors='coerce')
df['montant_min'] = df['montant_min'].replace({pd.NA: None})
df['montant_max'] = df['montant_max'].replace({pd.NA: None})
df = df.where(pd.notnull(df), None)

for index, row in df.iterrows():
    insert_query = """
    INSERT INTO markets (
        annee_de_notification, numero_marche, objet_du_marche, nature_du_marche, fournisseur_nom,
        fournisseur_siret, fournisseur_code_postal, fournisseur_ville, montant_min, montant_max,
        date_de_notification, date_de_debut, date_de_fin, duree_du_marche, perimetre_financier, 
        categorie_d_achat_cle, categorie_d_achat_texte
    ) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    cursor.execute(insert_query, (
        row['annee_de_notification'], row['numero_marche'], row['objet_du_marche'], row['nature_du_marche'],
        row['fournisseur_nom'], row['fournisseur_siret'], row['fournisseur_code_postal'], row['fournisseur_ville'],
        row['montant_min'], row['montant_max'], row['date_de_notification'], row['date_de_debut'],
        row['date_de_fin'], row['duree_du_marche'], row['perimetre_financier'], row['categorie_d_achat_cle'],
        row['categorie_d_achat_texte']
    ))

connection.commit()
cursor.close()
connection.close()

('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment')
('annee_de_notification', 'year(4)', 'YES', '', None, '')
('numero_marche', 'varchar(50)', 'YES', '', None, '')
('objet_du_marche', 'text', 'YES', '', None, '')
('nature_du_marche', 'varchar(255)', 'YES', '', None, '')
('fournisseur_nom', 'varchar(255)', 'YES', '', None, '')
('fournisseur_siret', 'varchar(20)', 'YES', '', None, '')
('fournisseur_code_postal', 'varchar(20)', 'YES', '', None, '')
('fournisseur_ville', 'varchar(255)', 'YES', '', None, '')
('montant_min', 'decimal(15,2)', 'YES', '', None, '')
('montant_max', 'decimal(15,2)', 'YES', '', None, '')
('date_de_notification', 'date', 'YES', '', None, '')
('date_de_debut', 'date', 'YES', '', None, '')
('date_de_fin', 'date', 'YES', '', None, '')
('duree_du_marche', 'int(11)', 'YES', '', None, '')
('perimetre_financier', 'varchar(255)', 'YES', '', None, '')
('categorie_d_achat_cle', 'varchar(255)', 'YES', '', None, '')
('categorie_d_achat_texte', 'varchar(255)', 'YES', '', Non